In [1]:
!pip install -q --upgrade transformers peft trl datasets accelerate bitsandbytes scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
csv_path = "/content/drive/MyDrive/custom_data_rfu.csv"
# Cargar el archivo CSV
df = pd.read_csv(csv_path)

In [5]:
# Estratificación manual por quantiles del label (para regresión)
df['label_bin'] = pd.qcut(df['label'], q=10, duplicates='drop')
train_df, temp_df = train_test_split(df, test_size=0.19, stratify=df['label_bin'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=10/19, stratify=temp_df['label_bin'], random_state=42)

# Eliminar columna auxiliar
for d in [train_df, val_df, test_df]:
    d.drop(columns='label_bin', inplace=True)

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
# Guardar como .jsonl para usar con HuggingFace Datasets
def save_jsonl(df, filename):
    with open(filename, 'w') as f:
        for _, row in df.iterrows():
            prompt = f"Given the peptide sequence '{row['sequence']}', predict its activity value."
            response = str(row['label'])
            json.dump({"prompt": prompt, "response": response}, f)
            f.write('\n')

save_jsonl(train_df, "/content/drive/MyDrive/train.jsonl")
save_jsonl(val_df, "/content/drive/MyDrive/val.jsonl")
save_jsonl(test_df, "/content/drive/MyDrive/test.jsonl")

In [9]:
model_id = "meta-llama/Llama-3.2-1B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

train_dataset = load_dataset("json", data_files="/content/drive/MyDrive/train.jsonl", split="train")
val_dataset = load_dataset("json", data_files="/content/drive/MyDrive/val.jsonl", split="train")

def formatting_func(example):
    return {
        "prompt": example['prompt'],
        "completion": example['response']
    }

train_dataset = train_dataset.map(formatting_func)
val_dataset = val_dataset.map(formatting_func)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

#entrenamiento
training_args = TrainingArguments(
    output_dir="./llama3-regression",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=50,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
    peft_config=lora_config
)


trainer.train()


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Tokenizing eval dataset:   0%|          | 0/43 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/43 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,4.357100
1000,3.570700
1500,3.536800
2000,3.513200


TrainOutput(global_step=2400, training_loss=3.70346923828125, metrics={'train_runtime': 1531.3703, 'train_samples_per_second': 12.505, 'train_steps_per_second': 1.567, 'total_flos': 3030943717232640.0, 'train_loss': 3.70346923828125})

In [10]:
#Evaluación sobre test
test_dataset = load_dataset("json", data_files="/content/drive/MyDrive/test.jsonl", split="train")
import numpy as np

def predict(sequence):
    prompt = f"Given the peptide sequence '{sequence}', predict its activity value."
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=10)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return float(result.replace(prompt, "").strip())

preds = []
truth = []

for row in test_dataset:
    try:
        y_pred = predict(row['prompt'].split("'")[1])
        y_true = float(row['response'])
        preds.append(y_pred)
        truth.append(y_true)
    except:
        continue



Generating train split: 0 examples [00:00, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print("Test RMSE:", np.sqrt(mean_squared_error(truth, preds)))
print("Test MAE:", mean_absolute_error(truth, preds))
print("R^2 Score:", r2_score(truth, preds))

Test RMSE: 0.5555973639132906
Test MAE: 0.4253509115625
R^2 Score: 0.24819291863619042


In [12]:
output_model_dir = "/content/drive/MyDrive/ELVA/CLASIFICADOR/llama3-regression-model"

# Guardar el modelo entrenado con LoRA
trainer.save_model(output_model_dir)

# Guardar el tokenizador
tokenizer.save_pretrained(output_model_dir)

('/content/drive/MyDrive/ELVA/CLASIFICADOR/llama3-regression-model/tokenizer_config.json',
 '/content/drive/MyDrive/ELVA/CLASIFICADOR/llama3-regression-model/special_tokens_map.json',
 '/content/drive/MyDrive/ELVA/CLASIFICADOR/llama3-regression-model/tokenizer.json')